In [1]:
# This notebook is a modification of https://github.com/tensorflow/privacy/blob/master/tutorials/Classification_Privacy.ipynb

##### Copyright 2019 The TensorFlow Authors.


In [2]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Implement Differential Privacy with TensorFlow Privacy

<table class="tfo-notebook-buttons" align="left">
  
  <td>
    <a target="_blank" href="https://github.com/tensorflow/privacy/blob/master/tutorials/Classification_Privacy.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

## Overview

This tutorial uses [tf.keras](https://www.tensorflow.org/guide/keras) to train a convolutional neural network (CNN) to recognize handwritten digits with the SGD optimizer provided by the TensorFlow Privacy library.

## Setup

First, set this notebook's runtime to use a GPU, under Runtime > Change runtime type > Hardware accelerator. Then, begin importing the necessary libraries.

In [3]:
try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 1.x
except Exception:
    pass

import tensorflow as tf

import numpy as np

#tf.compat.v1.logging.set_verbosity(tf.logging.ERROR)

In [4]:
tf.__version__

'2.2.0'

In [5]:
#  %tensorflow_version 2.x

Install TensorFlow Privacy.

## Load and pre-process the dataset

Load the [MNIST](http://yann.lecun.com/exdb/mnist/) dataset and prepare the data for training.

In [6]:
train, test = tf.keras.datasets.mnist.load_data()
train_data, train_labels = train
test_data, test_labels = test

train_data = np.array(train_data, dtype=np.float32) / 255
test_data = np.array(test_data, dtype=np.float32) / 255

train_data = train_data.reshape(train_data.shape[0], 28, 28, 1)
test_data = test_data.reshape(test_data.shape[0], 28, 28, 1)

train_labels = np.array(train_labels, dtype=np.int32)
test_labels = np.array(test_labels, dtype=np.int32)

train_labels = tf.keras.utils.to_categorical(train_labels, num_classes=10)
test_labels = tf.keras.utils.to_categorical(test_labels, num_classes=10)

assert train_data.min() == 0.
assert train_data.max() == 1.
assert test_data.min() == 0.
assert test_data.max() == 1.

## Define and tune learning model hyperparameters
Set learning model hyperparamter values. 


In [7]:
epochs = 15
batch_size = 250

one existing hyperamater that you must tune:

1. `learning_rate` (float) - This hyperparameter already exists in vanilla SGD. The higher the learning rate, the more each update matters. If the updates are noisy (such as when the additive noise is large compared to the clipping threshold), a low learning rate may help the training procedure converge. 

Use the hyperparameter values below to obtain a reasonably accurate model (95% test accuracy):

In [8]:
learning_rate = 0.25


## Build the learning model

Define a convolutional neural network as the learning model. 

In [9]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16, 8,
                           strides=2,
                           padding='same',
                           activation='relu',
                           input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPool2D(2, 1),
    tf.keras.layers.Conv2D(32, 4,
                           strides=2,
                           padding='valid',
                           activation='relu'),
    tf.keras.layers.MaxPool2D(2, 1),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

Define the optimizer and loss function for the learning model. Compute the loss as a vector of losses per-example rather than as the mean over a minibatch to support gradient manipulation over each training point. 

In [10]:
optimizer = tf.compat.v1.train.GradientDescentOptimizer(learning_rate=learning_rate)

In [11]:
loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True, reduction=tf.losses.Reduction.NONE)

## Compile and train the learning model


In [13]:
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

history = model.fit(train_data, train_labels, epochs=epochs, validation_data=(test_data, test_labels), batch_size=batch_size)

Epoch 1/15
240/240 [==============================] - 7s 28ms/step - loss: 2.3619 - accuracy: 0.0993 - val_loss: 2.3579 - val_accuracy: 0.1032
Epoch 2/15
240/240 [==============================] - 7s 28ms/step - loss: 2.3619 - accuracy: 0.0993 - val_loss: 2.3579 - val_accuracy: 0.1032
Epoch 3/15
240/240 [==============================] - 7s 28ms/step - loss: 2.3619 - accuracy: 0.0993 - val_loss: 2.3579 - val_accuracy: 0.1032
Epoch 4/15
240/240 [==============================] - 7s 28ms/step - loss: 2.3619 - accuracy: 0.0993 - val_loss: 2.3579 - val_accuracy: 0.1032
Epoch 5/15
240/240 [==============================] - 7s 28ms/step - loss: 2.3619 - accuracy: 0.0993 - val_loss: 2.3579 - val_accuracy: 0.1032
Epoch 6/15
240/240 [==============================] - 7s 29ms/step - loss: 2.3619 - accuracy: 0.0993 - val_loss: 2.3579 - val_accuracy: 0.1032
Epoch 7/15
240/240 [==============================] - 7s 29ms/step - loss: 2.3619 - accuracy: 0.0993 - val_loss: 2.3579 - val_accuracy: 0.1032

In [14]:
print('\nhistory dict:', history.history)


history dict: {'loss': [2.3618500232696533, 2.3618509769439697, 2.3618507385253906, 2.3618509769439697, 2.3618507385253906, 2.3618509769439697, 2.3618509769439697, 2.3618507385253906, 2.3618509769439697, 2.3618507385253906, 2.3618509769439697, 2.3618509769439697, 2.3618507385253906, 2.3618507385253906, 2.3618505001068115], 'accuracy': [0.09929999709129333, 0.09929999709129333, 0.09929999709129333, 0.09929999709129333, 0.09929999709129333, 0.09929999709129333, 0.09929999709129333, 0.09929999709129333, 0.09929999709129333, 0.09929999709129333, 0.09929999709129333, 0.09929999709129333, 0.09929999709129333, 0.09929999709129333, 0.09929999709129333], 'val_loss': [2.357949733734131, 2.357949733734131, 2.357949733734131, 2.357949733734131, 2.357949733734131, 2.357949733734131, 2.357949733734131, 2.357949733734131, 2.357949733734131, 2.357949733734131, 2.357949733734131, 2.357949733734131, 2.357949733734131, 2.357949733734131, 2.357949733734131], 'val_accuracy': [0.10320000350475311, 0.103200